In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt
import glob
import numpy as np
import xarray as xr
import pyart
import cartopy.crs as ccrs
import tqdm


## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119



In [2]:
def last_day_of_month(any_day):
    # get close to the end of the month for any day, and add 4 days 'over'
    next_month = any_day.replace(day=28) + dt.timedelta(days=4)
    # subtract the number of remaining 'overage' days to get last day of current month, or said programattically said, the previous day of the first of next month
    return next_month - dt.timedelta(days=next_month.day)

def plot_gust(rid, time, stn, ax):
    time2 = last_day_of_month(time)
    fid1 = time.replace(day=1).strftime("%Y%m%d")
    fid2 = time2.strftime("%Y%m%d")
    df=pd.read_pickle("/g/data/eg3/ab4502/ExtremeWind/points/era5_aws_tint_"+rid+"_"+fid1+"_"+fid2+"_max.pkl")
    df["dt_utc"] = pd.DatetimeIndex(df["dt_utc"])
    df.query("stn_id=="+str(stn)).loc[(df["dt_utc"]>=(time+dt.timedelta(hours=-6))) & (df["dt_utc"]<=(time+dt.timedelta(hours=6)))]\
                    .set_index("dt_utc").gust.plot(marker="x",ax=ax)

def plot_refl(time,rid,lat,lon,group_id,ax):
    f=xr.open_dataset("/g/data/rq0/level_2/"+rid+"/COLUMNMAXREFLECTIVITY/"+rid+"_"+time.strftime("%Y%m%d")+"_columnmaxreflectivity.nc")\
                .interp({"time":dt.datetime.strptime(group_id[0:14],"%Y%m%d%H%M%S")},method="nearest")
    refl = f.columnmaxreflectivity.values
    x = f.longitude.values
    y = f.latitude.values
    
    plt.contourf(x,y,refl,levels=np.linspace(0,70,21),cmap=pyart.graph.cm_colorblind.HomeyerRainbow)    
    cb=plt.colorbar()
    cb.set_label("Column maximum reflectivity (dBZ)")
    ax.plot(lon, lat, marker="^", mew=2, ms=10, color="lightgray", mec="k")
    ax.coastlines(lw=2,ls="-")
    g1=ax.gridlines(draw_labels=True,ls=":"); g1.right_labels=False; g1.top_labels=False
    plt.contour(x,y,refl>30,colors="k",linewidths=0.25)
    plt.title(f.time.values)
    
def plot_driver(rid,state,display=True):
    scw_events = pd.read_csv("/g/data/eg3/ab4502/ExtremeWind/points/"+rid+"_scw_envs_df.csv")
    stn_info = load_stn_info(state)
    print(str(scw_events.shape[0]) + " SCW events...")
    for name, row in tqdm.tqdm(scw_events.iterrows()):
        plt.figure(figsize=[16,8])
        plt.suptitle(row.dt_utc+"\n"+str(row.gust)+" m/s"+"\n"+"Station id: "+str(row.stn_id))            
        plot_gust(rid,pd.to_datetime(row.dt_utc),row.stn_id,plt.subplot(1,2,1))
        plot_refl(pd.to_datetime(row.dt_utc),rid,stn_info.query("stn_no=="+str(row.stn_id)).lat,stn_info.query("stn_no=="+str(row.stn_id)).lon
                  ,row.group_id,plt.subplot(1,2,2,projection=ccrs.PlateCarree()))
        plt.savefig("/g/data/eg3/ab4502/figs/scw_figs_wgr2/"+rid+"_"+pd.to_datetime(row.dt_utc).strftime("%Y-%m-%d_%H:%M")+".png",bbox_inches="tight")
        if display:
            pass
        else:
            plt.close()

def load_stn_info(state):

        #Load station info
        names = ["id", "stn_no", "district", "stn_name", "site_open", "site_close", "lat", "lon", "latlon_method", "state",\
                        "hgt_asl", "hgt_asl_baro", "wmo_idx", "y1", "y2", "comp%", "Y%", "N%", "W%", "S%", "I%", "#"]

        if state == "vic_nsw":
                stn_info1 = pd.read_csv(glob.glob("/g/data/eg3/ab4502/ExtremeWind/obs/aws/vic_one_min_gust/HD01D_StnDet_*.txt")[0],\
                        names=names, header=0)
                stn_info2 = pd.read_csv(glob.glob("/g/data/eg3/ab4502/ExtremeWind/obs/aws/nsw_one_min_gust/HD01D_StnDet_*.txt")[0],\
                        names=names, header=0)
                stn_info = pd.concat([stn_info1, stn_info2], axis=0)
        elif state=="nt":
                stn_info = pd.concat(\
                    [pd.read_csv(f, names=names, header=None) for f in glob.glob("/g/data/eg3/ab4502/ExtremeWind/obs/aws/nt_one_min_gust/HD01D_StnDet_*.txt")],axis=0).\
                    sort_values("stn_name")
        elif state=="tas":
                stn_info = pd.concat(\
                    [pd.read_csv(f, names=names, header=None) for f in glob.glob("/g/data/eg3/ab4502/ExtremeWind/obs/aws/tas_one_min_gust/HD01D_StnDet_*.txt")],axis=0).\
                    sort_values("stn_name")
        else:
                stn_info = pd.read_csv(glob.glob("/g/data/eg3/ab4502/ExtremeWind/obs/aws/"+state+"_one_min_gust/HD01D_StnDet_*.txt")[0],\
                        names=names, header=0)
        return stn_info


In [4]:
#Darwin
plot_driver("63","nt",display=False)

36 SCW events...


36it [00:48,  1.34s/it]


In [5]:
#Hobart
plot_driver("76","tas",display=False)

45 SCW events...


45it [00:53,  1.20s/it]


In [6]:
#Melbourne
plot_driver("2","vic",display=False)

85 SCW events...


85it [04:22,  3.09s/it]


In [7]:
#Newcastle
plot_driver("4","nsw",display=False)

18 SCW events...


18it [00:40,  2.24s/it]


In [8]:
#Gympie
plot_driver("8","qld",display=False)

20 SCW events...


20it [00:36,  1.81s/it]


In [9]:
#Cairns
plot_driver("19","qld",display=False)

0 SCW events...


0it [00:00, ?it/s]


In [10]:
#Canberra
plot_driver("40","nsw",display=False)

10 SCW events...


10it [00:19,  1.98s/it]


In [11]:
#Kalgoorlie
plot_driver("48","wa",display=False)

4 SCW events...


4it [00:03,  1.06it/s]


In [12]:
#Yarrawonga
plot_driver("49","vic_nsw",display=False)

21 SCW events...


21it [00:36,  1.76s/it]


In [13]:
#Adelaide
plot_driver("64","sa",display=False)

26 SCW events...


26it [01:24,  3.24s/it]


In [14]:
#Brisbane
plot_driver("66","qld",display=False)

24 SCW events...


24it [00:50,  2.10s/it]


In [15]:
#Bairnsdale
plot_driver("68","vic",display=False)

8 SCW events...


8it [00:16,  2.05s/it]


In [16]:
#Namoi
plot_driver("69","nsw",display=False)

49 SCW events...


49it [01:02,  1.27s/it]


In [17]:
#Perth
plot_driver("70","wa",display=False)

22 SCW events...


22it [00:45,  2.09s/it]


In [18]:
#Sydney
plot_driver("71","nsw",display=False)

33 SCW events...


33it [01:50,  3.34s/it]


In [19]:
#Emerald
plot_driver("72","qld",display=False)

12 SCW events...


12it [00:12,  1.00s/it]


In [20]:
#Townsville
plot_driver("73","qld",display=False)

3 SCW events...


3it [00:04,  1.52s/it]


In [21]:
#Mt. Isa
plot_driver("75","qld",display=False)

14 SCW events...


14it [00:21,  1.53s/it]


In [22]:
#Weipa
plot_driver("78","qld",display=False)

1 SCW events...


1it [00:01,  1.17s/it]
